# ETL Notebook
"Reads and processes a single file from song_data and log_data and loads the data into your tables. This notebook contains detailed instructions on the ETL process for each of the tables"

In [88]:
import os
import glob
import psycopg2
from sql_queries import *
import pandas as pd

Configuring connection with postgres db

In [89]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=sparkify password=pass123")
cur = conn.cursor()
# conn.close()

Fetch data files

In [90]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process song_data
In this first part, we'll perform ETL on the first dataset, song_data, to create the songs and artists dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.

* Use the get_files function provided above to get a list of all song JSON files in data/song_data
* Select the first song in this list
* Read the song file and view the data

In [91]:
song_files = get_files("data/data_files/song_data/")
print(song_files)

['g:\\GitHub\\Data_Modeling_with_Postgres\\data\\data_files\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json', 'g:\\GitHub\\Data_Modeling_with_Postgres\\data\\data_files\\song_data\\A\\A\\A\\TRAAABD128F429CF47.json', 'g:\\GitHub\\Data_Modeling_with_Postgres\\data\\data_files\\song_data\\A\\A\\A\\TRAAADZ128F9348C2E.json', 'g:\\GitHub\\Data_Modeling_with_Postgres\\data\\data_files\\song_data\\A\\A\\A\\TRAAAEF128F4273421.json', 'g:\\GitHub\\Data_Modeling_with_Postgres\\data\\data_files\\song_data\\A\\A\\A\\TRAAAFD128F92F423A.json', 'g:\\GitHub\\Data_Modeling_with_Postgres\\data\\data_files\\song_data\\A\\A\\A\\TRAAAMO128F1481E7F.json', 'g:\\GitHub\\Data_Modeling_with_Postgres\\data\\data_files\\song_data\\A\\A\\A\\TRAAAMQ128F1460CD3.json', 'g:\\GitHub\\Data_Modeling_with_Postgres\\data\\data_files\\song_data\\A\\A\\A\\TRAAAPK128E0786D96.json', 'g:\\GitHub\\Data_Modeling_with_Postgres\\data\\data_files\\song_data\\A\\A\\A\\TRAAARJ128F9320760.json', 'g:\\GitHub\\Data_Modeling_with_Postgres\\dat

In [92]:
filepath = song_files[0]

Reading data into dataframe

In [93]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


## Songs Table
# Extract Data for Songs Table
* Select columns for song ID, title, artist ID, year, and duration
* Use df.values to select just the values from the dataframe
* Index to select the first (only) record in the dataframe
* Convert the array to a list and set it to song_data

In [94]:
song_data = df[['song_id', 'title','artist_id','year','duration']].values.tolist()[0]

# Insert Record into Song Table
Implement the song_table_insert query in sql_queries.py and run the cell below to insert a record for this song into the songs table. Remember to run create_tables.py before running the cell below to ensure you've created/resetted the songs table in the sparkify database.

In [95]:
cur.execute(songs_insert, song_data)
conn.commit()

## Artists Table
# Extract Data for Artists Table
* Select columns for artist ID, name, location, latitude, and longitude
* Use df.values to select just the values from the dataframe
* Index to select the first (only) record in the dataframe
* Convert the array to a list and set it to artist_data

In [96]:
artist_data = df[['artist_id', 'artist_name','artist_location','artist_latitude','artist_longitude']].values.tolist()[0]
print(artist_data)

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]


# Insert Record into Artist Table
Implement the artist_table_insert query in sql_queries.py and run the cell below to insert a record for this song's artist into the artists table. Remember to run create_tables.py before running the cell below to ensure you've created/resetted the artists table in the sparkify database.

In [97]:
cur.execute(artists_insert, artist_data)
conn.commit()

Run test.ipynb to see if you've successfully added a record to this table.

# Process log_data
In this part, you'll perform ETL on the second dataset, log_data, to create the time and users dimensional tables, as well as the songplays fact table.

Let's perform ETL on a single log file and load a single record into each table.

* Use the get_files function provided above to get a list of all log JSON files in data/log_data
* Select the first log file in this list
* Read the log file and view the data

In [98]:
log_files = get_files("data/data_files/logs")

In [99]:
filepath = log_files[0]

In [100]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## Time Table
# Extract Data for Time Table
* Filter records by NextSong action
* Convert the ts timestamp column to datetime
* * Hint: the current timestamp is in milliseconds
* Extract the timestamp, hour, day, week of year, month, year, and weekday from the ts column and set time_data to a list containing these values in order
* * Hint: use pandas' dt attribute to access easily datetimelike properties.
* Specify labels for these columns and set to column_labels
* Create a dataframe, time_df, containing the time data for this file by combining column_labels and time_data into a dictionary and converting this into a dataframe

In [101]:
df = pd.read_json(filepath, lines=True)
next_song = df['page'] == "NextSong"
df[next_song].head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [102]:
t = df[next_song].copy(deep=True)
t['ts'] = pd.to_datetime(df['ts'])
t.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1970-01-01 00:25:41.106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1970-01-01 00:25:41.106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1970-01-01 00:25:41.106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1970-01-01 00:25:41.106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1970-01-01 00:25:41.107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [103]:
time_data = ([t['ts'], t['ts'].dt.hour, t['ts'].dt.day, t['ts'].dt.isocalendar().week, t['ts'].dt.month, t['ts'].dt.year, t['ts'].dt.day_name() ])
column_labels = (['timestamp', 'hour','day', 'week_of_year','month', 'year', 'weekday'])

In [104]:
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
time_df.head()

,timestamp,hour,day,week_of_year,month,year,weekday
2,1970-01-01 00:25:41.106106796,0,1,1,1,1970,Thursday
4,1970-01-01 00:25:41.106352796,0,1,1,1,1970,Thursday
5,1970-01-01 00:25:41.106496796,0,1,1,1,1970,Thursday
6,1970-01-01 00:25:41.106673796,0,1,1,1,1970,Thursday
7,1970-01-01 00:25:41.107053796,0,1,1,1,1970,Thursday


# Insert Records into Time Table
Implement the time_table_insert query in sql_queries.py and run the cell below to insert records for the timestamps in this log file into the time table. Remember to run create_tables.py before running the cell below to ensure you've created/resetted the time table in the sparkify database.

In [105]:
for i, row in time_df.iterrows():
    cur.execute(time_insert, list(row))
    conn.commit()

Run test.ipynb to see if you've successfully added records to this table.

## Users Table
# Extract Data for Users Table
* Select columns for user ID, first name, last name, gender and level and set to user_df

In [106]:
user_df = t.filter(['userId','firstName','lastName','gender','level']).drop_duplicates()
user_df.head()

,userId,firstName,lastName,gender,level
2,8,Kaylee,Summers,F,free
10,10,Sylvie,Cruz,F,free
12,26,Ryan,Smith,M,free
14,101,Jayden,Fox,M,free


# Insert Records into Users Table
Implement the user_table_insert query in sql_queries.py and run the cell below to insert records for the users in this log file into the users table. Remember to run create_tables.py before running the cell below to ensure you've created/resetted the users table in the sparkify database.

In [107]:
for i, row in user_df.iterrows():
    cur.execute(users_insert, row)
    conn.commit()

Run test.ipynb to see if you've successfully added records to this table.

## SongPlays Table
# Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the songplays table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.

* Implement the song_select query in sql_queries.py to find the song ID and artist ID based on the title, artist name, and duration of a song.
* Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to songplay_data
# Insert Records into Songplays Table
* Implement the songplay_table_insert query and run the cell below to insert records for the songplay actions in this log file into the songplays table. Remember to run create_tables.py before running the cell below to ensure you've created/resetted the songplays table in the sparkify database.

In [108]:
for index, row in df.iterrows():
    
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    # insert songplay record
    songplay_data = ([pd.to_datetime(row.ts), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent])
    cur.execute(songplays_insert, songplay_data)
    conn.commit()

Run test.ipynb to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [109]:
conn.close()